In [1]:
import math

import numpy as np
import pyproj
import vtk

In [2]:
rt90 = 'epsg:3021'  # RT90
wgs84 = 'epsg:4326'  # Global lat-lon coordinate system

rt90_to_latlon = pyproj.Transformer.from_crs(rt90, wgs84)
tl = rt90_to_latlon.transform(1349340, 7022573)
tr = rt90_to_latlon.transform(1371573, 7022967)
br = rt90_to_latlon.transform(1371835, 7006362)
bl = rt90_to_latlon.transform(1349602, 7005969)

In [3]:
px = [bl[0], br[0], tr[0], tl[0]]
py = [bl[1], br[1], tr[1], tl[1]]

# compute coefficients
A = [[1, 0, 0, 0], [1, 1, 0, 0], [1, 1, 1, 1], [1, 0, 1, 0]]
AI = np.linalg.inv(A)
a = np.matmul(AI,px)
b = np.matmul(AI,py)

In [4]:
a

array([ 8.69060359e+00,  1.41589334e-01, -1.77190387e-02, -2.66615436e-04])

In [5]:
b

array([ 6.06848165e+01,  2.43023050e-02,  1.06223200e-01, -2.59658596e-06])

In [6]:
def LtoX(x,y):
    #quadratic equation coeffs, aa*mm^2+bb*m+cc=0
    aa = a[3]*b[2] - a[2]*b[3]
    bb = a[3]*b[0] - a[0]*b[3] + a[1]*b[2] - a[2]*b[1] + x*b[3] - y*a[3]
    cc = a[1]*b[0] - a[0]*b[1] + x*b[1] - y*a[1]

    #compute m = (-b+sqrt(b^2-4ac))/(2a)
    det = math.sqrt(bb*bb - 4*aa*cc)
    m = (-bb+det)/(2*aa)

    #compute l
    l = (x-a[0]-a[2]*m)/(a[1]+a[3]*m)
    return l,m

In [7]:
def XtoL(l,m):
    newx = a[0] + a[1]*l + a[2]*m + a[3]*l*m
    newy = b[0] + b[1]*l + b[2]*m + b[3]*l*m
    return newx,newy

In [8]:
print(LtoX(bl[0],bl[1]))
print(LtoX(tl[0],tl[1]))
print(LtoX(br[0],br[1]))
print(LtoX(tr[0],tr[1]))

print(XtoL(0,0))
print(XtoL(0,1))
print(XtoL(1,0))
print(XtoL(1,1))

(0.0, -0.0)
(-1.5515484638678795e-14, 0.9999999999998762)
(1.0, -0.0)
(0.999999999999968, 0.9999999999998457)
(8.690603591508863, 60.68481654100474)
(8.672884552833715, 60.79103974072171)
(8.832192925809254, 60.709118845984214)
(8.81420727169797, 60.81533944911522)
